# Lab objective: Implementing Conversational AI Agent with Tools

### Installing necessary libraries

In [ ]:
!pip install langchain==0.1.11 langchain_community langchain_openai pinecone-client==3.1.0 pypdf faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchainhub

### Importing library and integrating API
The code sets environment variables for API keys for OpenAI and Tavily, ensuring these services can be accessed securely in the application.

It initializes an instance of TavilySearchResults, a tool for retrieving search results, presumably leveraging the Tavily API for specific functionalities.

In [ ]:
import os
os.environ["OPENAI_API_KEY"]= "OPENAI_API_KEY"
os.environ["TAVILY_API_KEY"]= "TAVILY_API_KEY"

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()


### Setting Up a Document Retrieval System with LangChain and FAISS

This code loads documents from a specified web URL using WebBaseLoader and then splits them into smaller, overlapping chunks using RecursiveCharacterTextSplitter to manage text size and overlap for processing.

It indexes these document chunks using FAISS (a high-performance vector search library) with OpenAI embeddings to create an efficient retriever for querying and retrieving relevant document sections.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=200).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()


# Making FAISS retriever and tavily search as tool

This tool is designed to search for information about LangSmith and is intended for answering questions related to LangSmith.

A list called "tools" that includes two elements: "search" and "retriever_tool“. By storing these tools in a list, they can be easily accessed and managed together within the program. 


In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
retriever,
name="langsmith_search",
description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
tools = [search, retriever_tool]


### Initializing OpenAI Chat Model and Retrieving Prompts with LangChain


This code initializes a LangChain-based chat model using OpenAI's GPT-3.5 Turbo and retrieves a specific prompt from a repository to guide the model's responses.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Get the prompt to use
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Setting Up and Executing a LangChain Agent for Processing Queries about LangSmith


This code sets up and executes a LangChain agent designed to respond to queries about LangSmith, integrating specific tools and prompts to process and answer user inputs effectively.

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "What is Langsmith and what for it is used?"})


{'input': 'What is Langsmith and what for it is used?',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith can be used independently without the need for LangChain. Users can install LangSmith, create an API key, set up their environment, log traces, and run evaluations using the platform. It provides tools for creating datasets, examples, evaluators, and more to enhance the development and evaluation process of LLM applications.'}